In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, GlobalAveragePooling2D, Reshape
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


In [3]:
# --- Load and preprocess hormone data ---
hormone_data = pd.read_csv('hormone_data.csv')
hormone_data.set_index('Patient File No.', inplace=True)

images, hormone_values, labels = [], [], []

base_path = 'images'

for folder in ['infected', 'not_infected']:
    folder_path = os.path.join(base_path, folder)
    label = 1 if folder == 'infected' else 0

    for img_file in os.listdir(folder_path):
        patient_id = int(img_file.replace('patient', '').replace('.jpg', ''))
        if patient_id in hormone_data.index:
            img_path = os.path.join(folder_path, img_file)
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img) / 255.0
            images.append(img_array)

            row = hormone_data.loc[patient_id].apply(pd.to_numeric, errors='coerce')
            row.fillna(row.median(), inplace=True)
            hormone_values.append(row.values)
            labels.append(label)

images = np.array(images)
hormone_values = np.array(hormone_values)
labels = np.array(labels)

X_train_img, X_test_img, X_train_tab, X_test_tab, y_train, y_test = train_test_split(
    images, hormone_values, labels, test_size=0.2, random_state=42
)

In [4]:
print("Infected:", np.sum(labels == 1))
print("Not Infected:", np.sum(labels == 0))

Infected: 336
Not Infected: 363


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import MobileNetV2
from sklearn.utils import class_weight


# --- Define image input and base model ---
image_input = Input(shape=(224, 224, 3))
mobilenet = MobileNetV2(include_top=False, input_tensor=image_input, weights='imagenet')
mobilenet.trainable = False  # Freeze base layers if using pretrained weights

# --- Image branch ---
x = mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Reshape((1, 128))(x)

# --- Hormone (tabular) branch ---
hormone_input = Input(shape=(X_train_tab.shape[1],))
h = Dense(128, activation='relu')(hormone_input)
h = Reshape((1, 128))(h)

# --- Fusion ---
fused = Concatenate(axis=1)([x, h])
fused = Flatten()(fused)
fused = Dense(64, activation='relu')(fused)
output = Dense(1, activation='sigmoid')(fused)

# --- Model definition ---
model = Model(inputs=[image_input, hormone_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- Class weight to handle imbalance ---
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# --- Early stopping ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --- Train ---
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [6]:
preds = model.predict([X_test_img, X_test_tab])
print("Raw predictions:", preds[:10].flatten())
print("Rounded predictions:", (preds[:10].flatten() > 0.5).astype(int))
print("True labels:", y_test[:10])

5/5 [==============================] - 4s 649ms/step
Raw predictions: [9.9998993e-01 7.0429325e-01 1.0000000e+00 3.1591887e-07 1.0000000e+00
 9.9999988e-01 7.6336921e-19 2.1018843e-26 4.7707781e-03 8.3125196e-09]
Rounded predictions: [1 1 1 0 1 1 0 0 0 0]
True labels: [1 0 0 1 1 1 1 1 1 0]


In [7]:
# Evaluate model performance
from sklearn.metrics import classification_report, confusion_matrix

y_pred_probs = model.predict([X_test_img, X_test_tab]).flatten()
y_pred = (y_pred_probs > 0.5).astype(int)

print("Raw predictions:", y_pred_probs)
print("Rounded predictions:", y_pred)
print("True labels:", y_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


5/5 [==============================] - 3s 646ms/step
Raw predictions: [9.99989927e-01 7.04293251e-01 1.00000000e+00 3.15918868e-07
 1.00000000e+00 9.99999881e-01 7.63369212e-19 2.10188429e-26
 4.77077812e-03 8.31251956e-09 9.89918470e-01 1.00000000e+00
 9.99699116e-01 1.73317576e-11 1.00000000e+00 1.28359607e-04
 1.00000000e+00 3.40977975e-04 1.00000000e+00 1.67452438e-31
 9.99999940e-01 9.99924064e-01 9.98910546e-01 1.65813631e-17
 1.00000000e+00 1.69552372e-16 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 7.64982654e-23 1.00000000e+00
 3.88885519e-06 9.99870181e-01 1.00000000e+00 1.03513048e-06
 1.00000000e+00 1.00000000e+00 9.94217634e-01 0.00000000e+00
 1.00000000e+00 2.99900239e-06 6.39065623e-01 1.00000000e+00
 1.00000000e+00 9.99998391e-01 9.99686003e-01 9.71770942e-01
 7.72285081e-19 1.00000000e+00 0.00000000e+00 2.22646151e-04
 2.45437011e-01 6.09911931e-21 0.00000000e+00 1.00000000e+00
 1.30583467e-02 1.00000000e+00 1.00000000e+00 1.72536186e-15
 1.00000000e+00

In [8]:
model.fit(
    [X_train_img, X_train_tab],
    y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.2,
    class_weight=class_weights_dict,
    callbacks=[early_stop]
)

Epoch 1/30
28/28 [==============================] - 20s 541ms/step - loss: 4.3962 - accuracy: 0.7069 - val_loss: 1.6189 - val_accuracy: 0.7321
Epoch 2/30
28/28 [==============================] - 21s 760ms/step - loss: 6.4391 - accuracy: 0.8859 - val_loss: 2.9976 - val_accuracy: 0.6696
Epoch 3/30
28/28 [==============================] - 24s 854ms/step - loss: 2.3530 - accuracy: 0.9262 - val_loss: 0.0522 - val_accuracy: 0.9643
Epoch 4/30
28/28 [==============================] - 23s 824ms/step - loss: 2.2454 - accuracy: 0.9776 - val_loss: 27.9312 - val_accuracy: 0.9286
Epoch 5/30
28/28 [==============================] - 23s 842ms/step - loss: 3.7820 - accuracy: 0.9799 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 6/30
28/28 [==============================] - 24s 849ms/step - loss: 5.0348 - accuracy: 0.9843 - val_loss: 0.7704 - val_accuracy: 0.9732
Epoch 7/30
28/28 [==============================] - 24s 848ms/step - loss: 2.2171 - accuracy: 0.9888 - val_loss: 0.0013 - val_accuracy: 1.000

In [9]:
model.save('vgg_model.keras')
